In [836]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA

In [838]:
def get_initial_11(team):
    base_path = '/Users/luisalawson/Desktop/Facultad/TD8/data'
    team_data_path = os.path.join(base_path, team, f'{team}_summary.csv')
    try:
        df = pd.read_csv(team_data_path)
        df = df.drop_duplicates()
        df = df.sort_values(by='MinutesPlayed', ascending=False)
        return df['PlayerName'][:11].tolist()
    except FileNotFoundError:
        print(f'Error: El archivo {team}_summary.csv no se encontró en la carpeta de {team}.')
        return None
    except Exception as e:
        print(f'Error al cargar el archivo para {team}: {str(e)}')
        return None

In [839]:
def get_team_players(team):
    base_path = '/Users/luisalawson/Desktop/Facultad/TD8/data'
    team_data_path = os.path.join(base_path, team, f'{team}_summary.csv')
    try:
        df = pd.read_csv(team_data_path)
        df = df.drop_duplicates()
        df = df.sort_values(by='MinutesPlayed', ascending=False)
        return df['PlayerName'].tolist()
    except FileNotFoundError:
        print(f'Error: El archivo {team}_summary.csv no se encontró en la carpeta de {team}.')
        return None
    except Exception as e:
        print(f'Error al cargar el archivo para {team}: {str(e)}')
        return None

In [840]:
def load_player_matrix(team_name, player_name, matrix_name):
    base_path = '/Users/luisalawson/Desktop/Facultad/TD8/data'
    if matrix_name != 'pass_transition_matrix':
        matrix_path = os.path.join(base_path, team_name, player_name, f'{matrix_name}.csv')
        matrix = np.loadtxt(matrix_path, delimiter=',')
    else:
        matrix_path = os.path.join(base_path, team_name, player_name, f'{matrix_name}.npy')
        matrix = np.load(matrix_path)
    return matrix

In [841]:
def aggregate_team_matrices(team, players):
    team_matrixes = {
        'pass_matrix':  np.zeros((10, 10)),
        'shot_matrix': np.zeros((10, 10)),
        'defense_matrix': np.zeros((10, 10)),
        'goal_matrix': np.zeros((10, 10)),
        'errors_matrix': np.zeros((10, 10)),
        'out_matrix': np.zeros((10, 10))
    }

    for player in players:
        team_matrixes['pass_matrix'] += load_player_matrix(team, player, 'pass_matrix')
        team_matrixes['shot_matrix'] += load_player_matrix(team, player, 'shot_matrix')
        team_matrixes['defense_matrix'] += load_player_matrix(team, player, 'defense_matrix')
        team_matrixes['goal_matrix'] += load_player_matrix(team, player, 'goal_matrix')
        team_matrixes['errors_matrix'] += load_player_matrix(team, player, 'errors_matrix')
        team_matrixes['out_matrix'] += load_player_matrix(team, player, 'out_matrix') 
    
    return team_matrixes

In [842]:
teams_list = ['Arsenal', 'Everton', 'Fulham', 'Manchester City',
       'Newcastle United', 'Queens Park Rangers', 'Reading',
       'West Bromwich Albion', 'West Ham United', 'Wigan Athletic',
       'Aston Villa', 'Chelsea', 'Liverpool', 'Manchester United',
       'Norwich City', 'Southampton', 'Stoke City', 'Sunderland',
       'Swansea City', 'Tottenham Hotspur']

In [843]:
base_path = '/Users/luisalawson/Desktop/Facultad/TD8/data'

team_matrixes = {}
player_matrixes = {}
all_players = []

for team in teams_list:
    team_csv_path = os.path.join(base_path, f"{team}/{team}_summary.csv")
    team_data = pd.read_csv(team_csv_path)
    
    players = get_initial_11(team) 
    for player in players:
        if player not in player_matrixes:
            player_matrixes[player] = {}
        
        types = {
           
            'pass_matrix':  np.zeros((10, 10)),
            'shot_matrix': np.zeros((10, 10)),
            'defense_matrix': np.zeros((10, 10)),
            'goal_matrix': np.zeros((10, 10)),
            'errors_matrix': np.zeros((10, 10)),
            'out_matrix': np.zeros((10, 10))
        }
        
        all_players.append(player)
        
        for type_name in types:
            
            player_matrixes[player][type_name] = load_player_matrix(team, player, type_name)
        
        player_info = team_data[team_data['PlayerName'] == player]
        if not player_info.empty:
            position = player_info['Position'].values[0]
            cards = player_info['Cards'].values[0]
            time_played = player_info['MinutesPlayed'].values[0]
            player_matrixes[player]['team'] = team
            player_matrixes[player]['position'] = position
            player_matrixes[player]['cards'] = cards
            player_matrixes[player]['time'] = time_played

    team_matrixes[team] = aggregate_team_matrices(team, players)

In [844]:
for team in teams_list:
    for matrix in team_matrixes[f'{team}']:
        team_matrixes[f'{team}'][matrix] = np.sum(team_matrixes[f'{team}'][matrix] * 90)

In [845]:
for player in all_players:
    for matrix_name, matrix_value in player_matrixes[f'{player}'].items():
        if matrix_name == 'cards':
            player_matrixes[f'{player}'][matrix_name] = matrix_value * 90
        elif matrix_name == 'time':
            player_matrixes[f'{player}'][matrix_name] = matrix_value / 90
        elif matrix_name == 'position' or matrix_name == 'team':
            pass
        else:
            player_matrixes[f'{player}'][matrix_name] = np.sum(matrix_value * 90)

In [846]:
player_dataframe = pd.DataFrame(player_matrixes).T

In [847]:
player_dataframe

,pass_matrix,shot_matrix,defense_matrix,goal_matrix,errors_matrix,out_matrix,team,position,cards,time
"Cazorla, Santiago",56.60181,3.02211,10.14408,0.31536,3.75804,2.81196,Arsenal,Midfielder,0.026277,38.055556
"Mertesacker, Per",43.94241,0.47952,10.52028,0.08451,4.31514,1.35297,Arsenal,Defender,0.112817,35.455556
"Arteta, Mikel",73.21698,0.55062,13.47732,0.17388,4.23117,1.5939,Arsenal,Midfielder,0.173913,34.5
"Giroud, Olivier",16.45299,3.92085,7.73109,0.40302,7.73145,3.11355,Arsenal,Striker,0.14658,27.288889
"Walcott, Theo",17.0568,3.28293,4.60359,0.5283,2.30157,2.15019,Arsenal,Striker,0.113208,26.5
...,...,...,...,...,...,...,...,...,...,...
"Dembele, Mousa",52.93458,0.91476,11.81214,0.03978,3.69936,1.71054,Tottenham Hotspur,Midfielder,0.159081,25.144444
"Dawson, Michael",38.25567,0.38313,16.21341,0.04257,9.10674,2.85156,Tottenham Hotspur,Defender,0.170213,23.5
"Dempsey, Clint",26.97858,3.03633,8.19837,0.30366,7.24419,2.55942,Tottenham Hotspur,Striker,0.173494,23.055556
"Guimaraes Cordeiro, Sandro Ranieri",39.14316,1.22004,15.56928,0.04878,6.19839,1.95174,Tottenham Hotspur,Midfielder,0.195228,20.488889


In [848]:
for team in teams_list:
    team_stats = team_matrixes[team]
    for player in get_initial_11(team):
        if player in player_dataframe.index:
            for stat_name, stat_value in team_stats.items():
                player_dataframe.loc[player, f'{stat_name}_team'] = stat_value

In [849]:
metrics = ['pass_matrix', 'shot_matrix', 'defense_matrix', 'goal_matrix', 'errors_matrix', 'out_matrix']

for metric in metrics:
    player_dataframe[f'{metric}_contribution'] = player_dataframe[metric] / player_dataframe[f'{metric}_team']

In [850]:
player_dataframe = player_dataframe.round(2)

In [851]:
player_dataframe

,pass_matrix,shot_matrix,defense_matrix,goal_matrix,errors_matrix,out_matrix,team,position,cards,time,...,defense_matrix_team,goal_matrix_team,errors_matrix_team,out_matrix_team,pass_matrix_contribution,shot_matrix_contribution,defense_matrix_contribution,goal_matrix_contribution,errors_matrix_contribution,out_matrix_contribution
"Cazorla, Santiago",56.60181,3.02211,10.14408,0.31536,3.75804,2.81196,Arsenal,Midfielder,0.026277,38.055556,...,102.83,2.03,47.92,25.71,0.126056,0.183224,0.098647,0.155085,0.078422,0.109374
"Mertesacker, Per",43.94241,0.47952,10.52028,0.08451,4.31514,1.35297,Arsenal,Defender,0.112817,35.455556,...,102.83,2.03,47.92,25.71,0.097863,0.029072,0.102305,0.04156,0.090047,0.052625
"Arteta, Mikel",73.21698,0.55062,13.47732,0.17388,4.23117,1.5939,Arsenal,Midfielder,0.173913,34.5,...,102.83,2.03,47.92,25.71,0.163059,0.033383,0.131061,0.085509,0.088295,0.061996
"Giroud, Olivier",16.45299,3.92085,7.73109,0.40302,7.73145,3.11355,Arsenal,Striker,0.14658,27.288889,...,102.83,2.03,47.92,25.71,0.036642,0.237713,0.075181,0.198194,0.161337,0.121104
"Walcott, Theo",17.0568,3.28293,4.60359,0.5283,2.30157,2.15019,Arsenal,Striker,0.113208,26.5,...,102.83,2.03,47.92,25.71,0.037987,0.199037,0.044768,0.259803,0.048028,0.083634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Dembele, Mousa",52.93458,0.91476,11.81214,0.03978,3.69936,1.71054,Tottenham Hotspur,Midfielder,0.159081,25.144444,...,104.29,1.97,55.59,27.36,0.159426,0.047336,0.113262,0.020176,0.066548,0.062515
"Dawson, Michael",38.25567,0.38313,16.21341,0.04257,9.10674,2.85156,Tottenham Hotspur,Defender,0.170213,23.5,...,104.29,1.97,55.59,27.36,0.115216,0.019826,0.155464,0.021591,0.163823,0.104215
"Dempsey, Clint",26.97858,3.03633,8.19837,0.30366,7.24419,2.55942,Tottenham Hotspur,Striker,0.173494,23.055556,...,104.29,1.97,55.59,27.36,0.081253,0.157121,0.078611,0.154015,0.130317,0.093539
"Guimaraes Cordeiro, Sandro Ranieri",39.14316,1.22004,15.56928,0.04878,6.19839,1.95174,Tottenham Hotspur,Midfielder,0.195228,20.488889,...,104.29,1.97,55.59,27.36,0.117889,0.063133,0.149288,0.024741,0.111504,0.07133


In [852]:
player_dataframe = player_dataframe.reset_index().rename(columns={'index': 'player'})

In [853]:
ranking = pd.read_excel('/Users/luisalawson/Facultad/TD8/soccer_defense_analysis/Ranking.xlsx')

In [854]:
player_dataframe = player_dataframe.merge(ranking[['Equipo', 'Posicion']], left_on='team', right_on='Equipo', how='left')
player_dataframe = player_dataframe.drop(columns=['Equipo'])
player_dataframe = player_dataframe.rename(columns={'Posicion': 'TeamRanking'})

In [855]:
player_dataframe.sort_values(by='goal_matrix_contribution', ascending=False)

,player,pass_matrix,shot_matrix,defense_matrix,goal_matrix,errors_matrix,out_matrix,team,position,cards,...,goal_matrix_team,errors_matrix_team,out_matrix_team,pass_matrix_contribution,shot_matrix_contribution,defense_matrix_contribution,goal_matrix_contribution,errors_matrix_contribution,out_matrix_contribution,TeamRanking
76,"Le Fondre, Adam",10.7406,3.0294,3.80052,0.66096,5.61816,2.36844,Reading,Striker,0.165239,...,1.21,57.54,28.99,0.047908,0.284185,0.041248,0.545252,0.097633,0.081707,19
135,"Suarez, Luis",27.29313,5.48226,5.24745,0.67428,3.95802,4.22145,Liverpool,Striker,0.29316,...,1.43,43.98,29.97,0.062889,0.338037,0.045742,0.471581,0.09,0.140874,7
87,"Lukaku, Romelu",17.99046,4.17492,5.55138,0.73161,6.58485,3.01176,West Bromwich Albion,Striker,0.129125,...,1.55,49.68,27.08,0.062777,0.326996,0.058375,0.470973,0.132534,0.111229,8
200,"Perez Cuesta, Miguel",25.61463,3.19464,8.30592,0.52281,6.36021,2.35287,Swansea City,Striker,0.17425,...,1.23,53.18,24.91,0.058549,0.253617,0.075257,0.425225,0.119603,0.094444,9
49,"Cabaye, Yohan",38.71809,2.4633,10.87974,0.24624,4.39236,4.06395,Newcastle United,Striker,0.328467,...,0.59,60.67,27.36,0.123132,0.250731,0.091282,0.419118,0.072402,0.148562,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,"Begovic, Asmir",0.0,0.0,0.0,0.0,0.0,0.0,Stoke City,Goalkeeper,0.0,...,0.83,53.45,32.66,0.0,0.0,0.0,0.0,0.0,0.0,13
16,"Distin, Sylvain",26.15166,0.51516,15.42384,0.0,6.81822,3.06108,Everton,Defender,0.060606,...,1.46,51.37,29.85,0.079776,0.029672,0.141534,0.0,0.132732,0.102561,6
181,"Cameron, Geoff",23.25141,0.26136,13.97736,0.0,5.12811,3.23406,Stoke City,Defender,0.16328,...,0.83,53.45,32.66,0.100051,0.027588,0.115075,0.0,0.09595,0.099031,13
182,"Whelan, Glenn",35.72388,0.70551,11.66094,0.0,4.53636,2.65392,Stoke City,Midfielder,0.134429,...,0.83,53.45,32.66,0.153719,0.074471,0.096004,0.0,0.084878,0.081267,13


In [856]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [857]:
X = player_dataframe.drop(columns=['player', 'team', 'TeamRanking'])
y = player_dataframe['TeamRanking']

In [858]:
player_dataframe

,player,pass_matrix,shot_matrix,defense_matrix,goal_matrix,errors_matrix,out_matrix,team,position,cards,...,goal_matrix_team,errors_matrix_team,out_matrix_team,pass_matrix_contribution,shot_matrix_contribution,defense_matrix_contribution,goal_matrix_contribution,errors_matrix_contribution,out_matrix_contribution,TeamRanking
0,"Cazorla, Santiago",56.60181,3.02211,10.14408,0.31536,3.75804,2.81196,Arsenal,Midfielder,0.026277,...,2.03,47.92,25.71,0.126056,0.183224,0.098647,0.155085,0.078422,0.109374,4
1,"Mertesacker, Per",43.94241,0.47952,10.52028,0.08451,4.31514,1.35297,Arsenal,Defender,0.112817,...,2.03,47.92,25.71,0.097863,0.029072,0.102305,0.04156,0.090047,0.052625,4
2,"Arteta, Mikel",73.21698,0.55062,13.47732,0.17388,4.23117,1.5939,Arsenal,Midfielder,0.173913,...,2.03,47.92,25.71,0.163059,0.033383,0.131061,0.085509,0.088295,0.061996,4
3,"Giroud, Olivier",16.45299,3.92085,7.73109,0.40302,7.73145,3.11355,Arsenal,Striker,0.14658,...,2.03,47.92,25.71,0.036642,0.237713,0.075181,0.198194,0.161337,0.121104,4
4,"Walcott, Theo",17.0568,3.28293,4.60359,0.5283,2.30157,2.15019,Arsenal,Striker,0.113208,...,2.03,47.92,25.71,0.037987,0.199037,0.044768,0.259803,0.048028,0.083634,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Dembele, Mousa",52.93458,0.91476,11.81214,0.03978,3.69936,1.71054,Tottenham Hotspur,Midfielder,0.159081,...,1.97,55.59,27.36,0.159426,0.047336,0.113262,0.020176,0.066548,0.062515,5
216,"Dawson, Michael",38.25567,0.38313,16.21341,0.04257,9.10674,2.85156,Tottenham Hotspur,Defender,0.170213,...,1.97,55.59,27.36,0.115216,0.019826,0.155464,0.021591,0.163823,0.104215,5
217,"Dempsey, Clint",26.97858,3.03633,8.19837,0.30366,7.24419,2.55942,Tottenham Hotspur,Striker,0.173494,...,1.97,55.59,27.36,0.081253,0.157121,0.078611,0.154015,0.130317,0.093539,5
218,"Guimaraes Cordeiro, Sandro Ranieri",39.14316,1.22004,15.56928,0.04878,6.19839,1.95174,Tottenham Hotspur,Midfielder,0.195228,...,1.97,55.59,27.36,0.117889,0.063133,0.149288,0.024741,0.111504,0.07133,5


In [859]:
X = pd.get_dummies(X, columns=['position'], prefix='position')

In [860]:
X = X.drop(columns = ['pass_matrix_team',
       'shot_matrix_team', 'defense_matrix_team', 'goal_matrix_team',
       'errors_matrix_team', 'out_matrix_team'])

In [861]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Random forest

In [862]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [863]:
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [864]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

Mean Squared Error: 22.311295454545455


## CNN

In [865]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Assuming X and y are already defined as per your setup
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=111)

# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),  # Dropout layer added
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),  # Another dropout layer
    layers.Dense(1)  # Assuming TeamRanking is continuous
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Training the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')



Epoch 1/100


/Users/luisalawson/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 161.2813 - mae: 11.4111 - val_loss: 116.0495 - val_mae: 9.5118
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 159.0651 - mae: 11.1900 - val_loss: 112.1250 - val_mae: 9.2970
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 148.5411 - mae: 10.8183 - val_loss: 108.3920 - val_mae: 9.0859
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 137.9620 - mae: 10.3374 - val_loss: 104.6994 - val_mae: 8.8700
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 129.4830 - mae: 10.0817 - val_loss: 100.9697 - val_mae: 8.6429
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 142.2057 - mae: 10.5589 - val_loss: 97.3358 - val_mae: 8.4126
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 127.6705 - mae: 9.8315 - val_loss: 93.6707 - val_mae: 8.1748
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 122.1891 - mae: 9.5780 - val_loss: 89.8201 - val_mae: 7.9167
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss:

In [866]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.1693 - mae: 4.9148  
Test MAE: 4.930480480194092


In [867]:
predictions = model.predict(X_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [868]:
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions.flatten()})

In [869]:
predictions_df

,Actual,Predicted
142,7,12.033096
178,13,14.296578
189,17,9.776166
55,20,12.377081
203,9,7.119573
...,...,...
87,8,10.594332
181,13,12.417564
116,15,9.694191
204,9,7.151748


In [870]:
team_rankings_pred = {}

for team in teams_list:
    players = get_initial_11(team)
    player_predictions = []

    for player in players:
        if player in player_matrixes:
            player_row = player_dataframe[player_dataframe['player'] == player]
            required_columns = [
                'pass_matrix',
                'shot_matrix',
                'defense_matrix',
                'goal_matrix',
                'errors_matrix',
                'out_matrix',
                'cards',
                'time',
                'pass_matrix_contribution',
                'shot_matrix_contribution',
                'defense_matrix_contribution',
                'goal_matrix_contribution',
                'errors_matrix_contribution',
                'out_matrix_contribution',
                'position',
                ]
            
            position_columns = ['position_Defender', 'position_Goalkeeper', 'position_Midfielder', 'position_Striker']
            
            for pos_col in position_columns:
                player_position = player_row['position'].values[0]
                if player_position == 'Striker':
                    player_row[pos_col] = 1 if pos_col == 'position_Striker' else 0
                elif player_position == 'Midfielder':
                    player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
                elif player_position == 'Defender':
                    player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
                elif player_position == 'Goalkeeper':
                    player_row[pos_col] = 1 if pos_col == 'position_Goalkeeper' else 0

            player_row = player_row[required_columns + position_columns]

            required_columns = [
                'pass_matrix',
                'shot_matrix',
                'defense_matrix',
                'goal_matrix',
                'errors_matrix',
                'out_matrix',
                'cards',
                'time',
                'pass_matrix_contribution',
                'shot_matrix_contribution',
                'defense_matrix_contribution',
                'goal_matrix_contribution',
                'errors_matrix_contribution',
                'out_matrix_contribution',
                'position_Defender', 'position_Goalkeeper', 'position_Midfielder', 'position_Striker'
               
                ]

            player_row = player_row.drop(columns=['position'])
            
            if all(col in player_row.columns for col in required_columns):
                player_features = player_row[required_columns].values
                player_features = np.array(player_features).reshape(1, -1)
                player_features = scaler.transform(player_features)
                player_prediction = model.predict(player_features)
                player_predictions.append(player_prediction[0])
            else:
                print(f"Missing columns for player {player}: {set(required_columns) - set(player_row.columns)}")
        else:
            print(f"Player {player} not found in player_matrixes")

    if player_predictions:
        team_rankings_pred[team] = np.mean(player_predictions)
    else:
        print(f"No predictions for team {team}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is t

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is tryin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is tryin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is tryin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [871]:
#create a df team and prediction
team_rankings_pred_df = pd.DataFrame(team_rankings_pred.items(), columns=['Team', 'PredictedRanking'])

In [872]:
team_rankings_pred_df.sort_values(by='PredictedRanking', ascending=True)

,Team,PredictedRanking
3,Manchester City,7.169022
18,Swansea City,7.780324
11,Chelsea,8.155096
13,Manchester United,8.281281
0,Arsenal,8.304034
12,Liverpool,8.691902
9,Wigan Athletic,9.068145
19,Tottenham Hotspur,9.109798
1,Everton,9.767823
7,West Bromwich Albion,9.898022


In [873]:

team_rankings_pred_df = team_rankings_pred_df.merge(ranking[['Equipo', 'Posicion']], left_on='Team', right_on='Equipo', how='left')

In [874]:
team_rankings_pred_df

,Team,PredictedRanking,Equipo,Posicion
0,Arsenal,8.304034,Arsenal,4
1,Everton,9.767823,Everton,6
2,Fulham,9.905080,Fulham,12
3,Manchester City,7.169022,Manchester City,2
4,Newcastle United,11.170434,Newcastle United,16
5,Queens Park Rangers,13.135552,Queens Park Rangers,20
6,Reading,14.052072,Reading,19
7,West Bromwich Albion,9.898022,West Bromwich Albion,8
8,West Ham United,11.292542,West Ham United,10
9,Wigan Athletic,9.068145,Wigan Athletic,18


In [875]:
team_rankings_pred_df['Real'] = team_rankings_pred_df['Posicion']
team_rankings_pred_df = team_rankings_pred_df.rename(columns={'PredictedRanking': 'Predicted'})
team_rankings_pred_df['Predicted'] = team_rankings_pred_df['Predicted'].rank(method='min')
team_rankings_pred_df['diff'] = team_rankings_pred_df['Real'] - team_rankings_pred_df['Predicted']
team_rankings_pred_df = team_rankings_pred_df[['Team', 'Predicted', 'Real', 'diff']]

In [876]:
team_rankings_pred_df.sort_values(by='Predicted', ascending=True)

,Team,Predicted,Real,diff
3,Manchester City,1.0,2,1.0
18,Swansea City,2.0,9,7.0
11,Chelsea,3.0,3,0.0
13,Manchester United,4.0,1,-3.0
0,Arsenal,5.0,4,-1.0
12,Liverpool,6.0,7,1.0
9,Wigan Athletic,7.0,18,11.0
19,Tottenham Hotspur,8.0,5,-3.0
1,Everton,9.0,6,-3.0
7,West Bromwich Albion,10.0,8,-2.0


## Generalized prediction

In [877]:
player_dataframe

,player,pass_matrix,shot_matrix,defense_matrix,goal_matrix,errors_matrix,out_matrix,team,position,cards,...,goal_matrix_team,errors_matrix_team,out_matrix_team,pass_matrix_contribution,shot_matrix_contribution,defense_matrix_contribution,goal_matrix_contribution,errors_matrix_contribution,out_matrix_contribution,TeamRanking
0,"Cazorla, Santiago",56.60181,3.02211,10.14408,0.31536,3.75804,2.81196,Arsenal,Midfielder,0.026277,...,2.03,47.92,25.71,0.126056,0.183224,0.098647,0.155085,0.078422,0.109374,4
1,"Mertesacker, Per",43.94241,0.47952,10.52028,0.08451,4.31514,1.35297,Arsenal,Defender,0.112817,...,2.03,47.92,25.71,0.097863,0.029072,0.102305,0.04156,0.090047,0.052625,4
2,"Arteta, Mikel",73.21698,0.55062,13.47732,0.17388,4.23117,1.5939,Arsenal,Midfielder,0.173913,...,2.03,47.92,25.71,0.163059,0.033383,0.131061,0.085509,0.088295,0.061996,4
3,"Giroud, Olivier",16.45299,3.92085,7.73109,0.40302,7.73145,3.11355,Arsenal,Striker,0.14658,...,2.03,47.92,25.71,0.036642,0.237713,0.075181,0.198194,0.161337,0.121104,4
4,"Walcott, Theo",17.0568,3.28293,4.60359,0.5283,2.30157,2.15019,Arsenal,Striker,0.113208,...,2.03,47.92,25.71,0.037987,0.199037,0.044768,0.259803,0.048028,0.083634,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,"Dembele, Mousa",52.93458,0.91476,11.81214,0.03978,3.69936,1.71054,Tottenham Hotspur,Midfielder,0.159081,...,1.97,55.59,27.36,0.159426,0.047336,0.113262,0.020176,0.066548,0.062515,5
216,"Dawson, Michael",38.25567,0.38313,16.21341,0.04257,9.10674,2.85156,Tottenham Hotspur,Defender,0.170213,...,1.97,55.59,27.36,0.115216,0.019826,0.155464,0.021591,0.163823,0.104215,5
217,"Dempsey, Clint",26.97858,3.03633,8.19837,0.30366,7.24419,2.55942,Tottenham Hotspur,Striker,0.173494,...,1.97,55.59,27.36,0.081253,0.157121,0.078611,0.154015,0.130317,0.093539,5
218,"Guimaraes Cordeiro, Sandro Ranieri",39.14316,1.22004,15.56928,0.04878,6.19839,1.95174,Tottenham Hotspur,Midfielder,0.195228,...,1.97,55.59,27.36,0.117889,0.063133,0.149288,0.024741,0.111504,0.07133,5


In [878]:
# if ranking is 1 to 4 -> 1
# if ranking is 5 to 8 -> 2
# if ranking is 9 to 12 -> 3
# if ranking is 13 to 16 -> 4
# if ranking is 17 to 20 -> 5
player_dataframe['TeamRanking'] = player_dataframe['TeamRanking'].apply(lambda x: 1 if x <= 4 else (2 if x <= 8 else (3 if x <= 12 else (4 if x <= 16 else 5))))

In [879]:
X = player_dataframe.drop(columns=['player', 'team', 'TeamRanking'])
y = player_dataframe['TeamRanking']

In [880]:
X = pd.get_dummies(X, columns=['position'], prefix='position')
X = X.drop(columns = ['pass_matrix_team',
       'shot_matrix_team', 'defense_matrix_team', 'goal_matrix_team',
       'errors_matrix_team', 'out_matrix_team'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [881]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

Mean Squared Error: 1.6433636363636366


In [882]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Assuming X and y are already defined as per your setup
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=111)

# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),  # Dropout layer added
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),  # Another dropout layer
    layers.Dense(1)  # Assuming TeamRanking is continuous
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Training the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/100


/Users/luisalawson/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 8.7898 - mae: 2.5899 - val_loss: 6.6080 - val_mae: 2.2131
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.6948 - mae: 2.5408 - val_loss: 5.7614 - val_mae: 2.0172
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.6451 - mae: 2.2428 - val_loss: 4.9810 - val_mae: 1.8231
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.8975 - mae: 2.2504 - val_loss: 4.2838 - val_mae: 1.6424
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.3786 - mae: 1.9094 - val_loss: 3.6961 - val_mae: 1.5019
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.6336 - mae: 1.7846 - val_loss: 3.2102 - val_mae: 1.3919
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.2040 - mae: 2.0830 - val_loss: 2.8299 - val_mae: 1.2942
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.9556 - mae: 1.8223 - val_loss: 2.5365 - val_mae: 1.2374
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.6726 - mae: 1.5339 - val_loss: 2

In [883]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2006 - mae: 1.2348  
Test MAE: 1.2588671445846558


In [884]:
team_rankings_pred = {}

for team in teams_list:
    players = get_initial_11(team)
    player_predictions = []

    for player in players:
        if player in player_matrixes:
            player_row = player_dataframe[player_dataframe['player'] == player]
            required_columns = [
                'pass_matrix',
                'shot_matrix',
                'defense_matrix',
                'goal_matrix',
                'errors_matrix',
                'out_matrix',
                'cards',
                'time',
                'pass_matrix_contribution',
                'shot_matrix_contribution',
                'defense_matrix_contribution',
                'goal_matrix_contribution',
                'errors_matrix_contribution',
                'out_matrix_contribution',
                'position',
                ]
            
            position_columns = ['position_Defender', 'position_Goalkeeper', 'position_Midfielder', 'position_Striker']
            
            for pos_col in position_columns:
                player_position = player_row['position'].values[0]
                if player_position == 'Striker':
                    player_row[pos_col] = 1 if pos_col == 'position_Striker' else 0
                elif player_position == 'Midfielder':
                    player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
                elif player_position == 'Defender':
                    player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
                elif player_position == 'Goalkeeper':
                    player_row[pos_col] = 1 if pos_col == 'position_Goalkeeper' else 0

            player_row = player_row[required_columns + position_columns]

            required_columns = [
                'pass_matrix',
                'shot_matrix',
                'defense_matrix',
                'goal_matrix',
                'errors_matrix',
                'out_matrix',
                'cards',
                'time',
                'pass_matrix_contribution',
                'shot_matrix_contribution',
                'defense_matrix_contribution',
                'goal_matrix_contribution',
                'errors_matrix_contribution',
                'out_matrix_contribution',
                'position_Defender', 'position_Goalkeeper', 'position_Midfielder', 'position_Striker'
               
                ]

            player_row = player_row.drop(columns=['position'])
            
            if all(col in player_row.columns for col in required_columns):
                player_features = player_row[required_columns].values
                player_features = np.array(player_features).reshape(1, -1)
                player_features = scaler.transform(player_features)
                player_prediction = model.predict(player_features)
                player_predictions.append(player_prediction[0])
            else:
                print(f"Missing columns for player {player}: {set(required_columns) - set(player_row.columns)}")
        else:
            print(f"Player {player} not found in player_matrixes")

    if player_predictions:
        team_rankings_pred[team] = np.mean(player_predictions)
    else:
        print(f"No predictions for team {team}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is t

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is tryin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


...

/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is t

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Defender' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:37: SettingWithCopyWarning: 
A value is tryin

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_row[pos_col] = 1 if pos_col == 'position_Midfielder' else 0
/var/folders/yh/ncj38dg95vnd5bhlqttk32lc0000gn/T/ipykernel_34639/4272711399.py:35: SettingWithCopyWarning: 
A value is t

In [885]:
#create a df team and prediction
team_rankings_pred_df = pd.DataFrame(team_rankings_pred.items(), columns=['Team', 'PredictedRanking'])

In [886]:
team_rankings_pred_df.sort_values(by='PredictedRanking', ascending=True)
team_rankings_pred_df = team_rankings_pred_df.merge(ranking[['Equipo', 'Posicion']], left_on='Team', right_on='Equipo', how='left')
team_rankings_pred_df['Real'] = team_rankings_pred_df['Posicion'].apply(lambda x: 1 if x <= 4 else (2 if x <= 8 else (3 if x <= 12 else (4 if x <= 16 else 5))))

team_rankings_pred_df = team_rankings_pred_df.rename(columns={'PredictedRanking': 'Predicted'})
team_rankings_pred_df['Predicted'] = (team_rankings_pred_df['Predicted'].rank(method='min')).apply(lambda x: 1 if x <= 4 else (2 if x <= 8 else (3 if x <= 12 else (4 if x <= 16 else 5))))

team_rankings_pred_df['diff'] = team_rankings_pred_df['Real'] - team_rankings_pred_df['Predicted']
team_rankings_pred_df = team_rankings_pred_df[['Team', 'Predicted', 'Real', 'diff']]

In [889]:
team_rankings_pred_df.sort_values(by='Predicted', ascending=True)

,Team,Predicted,Real,diff
13,Manchester United,1,1,0
3,Manchester City,1,1,0
11,Chelsea,1,1,0
18,Swansea City,1,3,2
0,Arsenal,2,1,-1
12,Liverpool,2,2,0
9,Wigan Athletic,2,5,3
19,Tottenham Hotspur,2,2,0
7,West Bromwich Albion,3,2,-1
2,Fulham,3,3,0
